Connect google drive

In [1]:
import tensorflow as tf
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT available")

# Get GPU device information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

GPU is available
Thu May 23 09:24:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------

In [2]:
!nvidia-smi

Thu May 23 09:24:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'1st Year Syllabus.pdf'
'3rd sem.pdf'
 AdmitCard-220310482612.pdf
'AI Assignment_2023-24.gdoc'
 AMS2610
 assignment-1.pdf
 assignment-2.pdf
 Autoencoders_Denoising_MNIST.ipynb
 canada_per_capita_income.csv
'Class 11 Science'
'class 12 marksheet '
'class 12.pdf'
 Classroom
 CNN_MNIST.ipynb
'Colab Notebooks'
'Copy of DSC_4815.JPG'
 CustomCNN_Leaves.ipynb
'data science'
'Data Science ineuron'
'DAV RK PURAM - PSE (1).docx'
'DAV RK PURAM - PSE.docx'
'Excel Practice.xlsx'
 Fawaz.pdf
'Fiza SOP.gdoc'
 gender_submission.csv
'GN2417-AIBEA-Registration-cum-Examination-Form (1).pdf'
 IMG_1360.MOV
 IMG_1368.MOV
 IMG_1369.MOV
 IMG_1370.MOV
 IMG_1371.MOV
 IMG_1372.MOV
 IMG_1373.MOV
 IMG_1384.MOV
 IMG_1386.MOV
 IMG20201022184225.jpg
 IMG20220327164419.jpg
 IMG20220327164516.jpg
 IMG20220327164517.jpg
 IMG20220327181613.jpg
 IMG-20230301-WA0010.jpg
 IMG_20231113_235247.jpg
 index.gdoc
 index.html
'LETTER HEAD.jpg'
'minor project'
'minor project code.ipynb'
'My Space'
 pse.ppt

**1.Cloning Darknet**

In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15851, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15851 (delta 5), reused 13 (delta 4), pack-reused 15833
Receiving objects: 100% (15851/15851), 14.42 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (10671/10671), done.


**2. Compile Darknet network for training YOLO V4**

In [6]:
#change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: un

**3. Configure Darknet network for training YOLO V4**

In [8]:
# Configuration file (cfg file)
!cp cfg/yolov4-tiny-3l.cfg cfg/detect.cfg

In [17]:
# Create a folder to save the weights
!mkdir "/mydrive/yolov4/"

mkdir: cannot create directory ‘/mydrive/yolov4/’: File exists


In [18]:
# Jet is a class name
!echo $'car\nbus\nvan\ntruck\nbike' > data/obj.names

!echo -e 'classes= 5\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/oj.names\nbackup = /mydrive/yolov4/' >data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [19]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2024-05-23 09:37:58--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  18.7MB/s    in 9.5s    

2024-05-23 09:38:08 (16.3 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
# Extracting custom object dataset
!unzip /mydrive/yolov4/traffic-detection-dataset.zip -d data/obj

Archive:  /mydrive/yolov4/traffic-detection-dataset.zip
replace data/obj/traffic-detection-dataset/2019-12-06 13-30-50 261.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: data/obj/traffic-detection-dataset/2019-12-06 13-30-50 261.jpg  
replace data/obj/traffic-detection-dataset/2019-12-08 09-02-50 261.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [13]:
import glob
images_list = glob.glob("data/obj/traffic-detection-dataset/*.jpg")
print(images_list)

['data/obj/traffic-detection-dataset/2019-12-08 10-06-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-09 13-14-51 261.jpg', 'data/obj/traffic-detection-dataset/2019-11-26 08-09-00 261.jpg', 'data/obj/traffic-detection-dataset/2019-10-15 19-22-00 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-08 09-12-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-10 10-32-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-09 08-40-51 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-19 08-50-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-07 06-52-51 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-08 17-26-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-06 14-12-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-08 12-58-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-10-20 10-01-00 261.jpg', 'data/obj/traffic-detection-dataset/2019-12-08 16-58-50 261.jpg', 'data/obj/traffic-detection-dataset/2019-10-19 08-40-00 261.jpg', 'data/obj

In [14]:
# Create training.txt file, required for YOLOV4 training
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6. Start The Training**

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/detect.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 44 Avg (IOU: 0.593653), count: 21, class_loss = 4347.211914, iou_loss = 82.002930, total_loss = 4429.214844 
 total_bbox = 135, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.649046), count: 1, class_loss = 271.431549, iou_loss = 0.060150, total_loss = 271.491699 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.567528), count: 3, class_loss = 1087.928711, iou_loss = 1.833374, total_loss = 1089.762085 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 44 Avg (IOU: 0.629665), count: 6, class_loss = 4337.840820, iou_loss = 16.314453, total_loss = 4354.155273 
 total_bbox = 145, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 270.806702, iou_loss = 0.000000, total_los